# ♻️ DataOps y CI/CD para Pipelines de Datos

Objetivo: implementar prácticas de DataOps con control de calidad, pruebas automatizadas, hooks de Git y pipelines de CI/CD (GitHub Actions) para flujos de datos.

- Duración: 90 min
- Dificultad: Media
- Prerrequisitos: Pytest básico, Git y GitHub

## 1. Pruebas de datos con Great Expectations y Pandera

In [ ]:
import pandas as pd
try:
    import great_expectations as ge
    from pandera import DataFrameSchema, Column, Check
    df = pd.DataFrame({
        'venta_id':[1,2,3],
        'total':[100.0, 50.0, 25.5],
        'metodo_pago':['tarjeta','cash','tarjeta']
    })
    # Great Expectations estilo rápido
    gdf = ge.from_pandas(df)
    gdf.expect_column_values_to_not_be_null('venta_id')
    gdf.expect_column_values_to_be_between('total', 0, 10000)
    print('GE checks:', gdf.validate().to_json_dict()['statistics'])
    # Pandera schema
    schema = DataFrameSchema({
        'venta_id': Column(int, Check.gt(0)),
        'total': Column(float, Check.ge(0)),
        'metodo_pago': Column(str)
    })
    schema.validate(df)
    print('Pandera OK')
except Exception as e:
    print('Instala great-expectations y pandera si deseas ejecutar este bloque:', e)

## 2. Pytest: estructura mínima

In [ ]:
# tests/test_transform.py
sample_code = r'''
# archivo: src/transform.py
def clean_total(x):
    try:
        v = float(x)
        return max(v, 0.0)
    except Exception:
        return 0.0

# archivo: tests/test_transform.py
from src.transform import clean_total
def test_clean_total():
    assert clean_total(10) == 10.0
    assert clean_total(-5) == 0.0
    assert clean_total('oops') == 0.0
'''
print(sample_code)

## 3. Pre-commit hooks (lint, format, tests rápidos)

In [ ]:
pre_commit_cfg = r'''
repos:
  - repo: https://github.com/psf/black
    rev: 22.6.0
    hooks:
      - id: black
  - repo: https://github.com/PyCQA/isort
    rev: 5.10.1
    hooks:
      - id: isort
  - repo: https://github.com/pycqa/flake8
    rev: 5.0.4
    hooks:
      - id: flake8
  - repo: local
    hooks:
      - id: pytest-quick
        name: pytest quick
        entry: pytest -q
        language: system
        types: [python]
'''
print(pre_commit_cfg)

## 4. GitHub Actions: CI para validar el repositorio

In [ ]:
gha_yaml = r'''
name: ci
on: [push, pull_request]
jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - uses: actions/setup-python@v4
        with:
          python-version: '3.10'
      - name: Install deps
        run: |
          pip install -r curso_ingenieria_datos/requirements.txt
      - name: Lint & Test
        run: |
          pip install pytest flake8 black
          flake8 .
          pytest -q
'''
print(gha_yaml)

## 5. Observabilidad: logs y métricas mínimas

In [ ]:
from loguru import logger
import time, random

def process_batch(n=5):
    for i in range(n):
        start = time.time()
        try:
            if random.random() < 0.1:
                raise ValueError('Fallo aleatorio')
            time.sleep(0.05)
            latency = time.time() - start
            logger.info(f'item={i} status=ok latency={latency:.3f}s')
        except Exception as e:
            logger.error(f'item={i} status=error err={e}')
process_batch(10)

## 6. Ejercicios

1. Agrega una regla de pre-commit que bloquee archivos > 2 MB.
2. Crea un workflow adicional que ejecute validaciones de datos con Great Expectations.
3. Añade cobertura de pruebas y publica un badge en el README.